## Code to check GPU integration


In [1]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for g in gpus:
            tf.config.experimental.set_memory_growth(g, True)
        print("Enabled memory growth for GPUs:", gpus)
    except Exception as e:
        print("Could not set memory growth:", e)
else:
    print("No GPUs found")


## Imports


In [ ]:
import pathlib, os
from pathlib import Path
import tensorflow as tf
from collections import Counter
import numpy as np
from tensorflow.keras import layers, models, optimizers, callbacks, regularizers
import datetime

# --- NEW IMPORTS FOR TRANSFER LEARNING ---
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

## Constants

In [ ]:
DATA_DIR = str("asl_alphabet_train")
IMG_SIZE = (200, 200)
BATCH_SIZE = 32
SEED = 42
VAL_SPLIT = 0.15

## Dataset imports

In [ ]:
# training dataset (subset="training") and validation dataset (subset="validation")
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VAL_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

## Class checks

In [ ]:
class_names = train_ds.class_names
print("Found classes:", len(class_names))
print(class_names)

In [ ]:
from collections import Counter
p = pathlib.Path(DATA_DIR)
counts = {d.name: len(list(d.glob("*"))) for d in p.iterdir() if d.is_dir()}
for c, n in sorted(counts.items()):
    print(f"{c:20s} {n}")


In [ ]:
type(train_ds)

In [ ]:
iterator = iter(train_ds)

first_batch = next(iterator)

images_batch, labels_batch = first_batch


print(f"Type of images_batch: {type(images_batch)}")
print(f"Images batch shape: {images_batch.shape}")
print(f"Labels batch shape: {labels_batch.shape}")

print("\n--- Example from the batch ---")
print(f"Shape of one image: {images_batch[0].shape}")
print(f"Label for first image: {labels_batch[0]}")

## Optimize pipeline (for Transfer Learning)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

# We can use the same augmentation as before
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomRotation(0.1),  
    tf.keras.layers.RandomZoom(0.1),    
    tf.keras.layers.RandomTranslation(height_factor=0.2, width_factor=0.2), 
    tf.keras.layers.RandomBrightness(0.1), 
    tf.keras.layers.RandomContrast(0.1)    
])

# --- Apply augmentation, THEN MobileNet's specific pre-processing ---
# This function scales pixels from [0, 255] to [-1, 1]
train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)

# --- Apply pre-processing to validation data ---
val_ds = val_ds.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
num_classes = len(class_names)
input_shape = (*IMG_SIZE, 3) 

# 1. Load the "Street Smart" Base Model (MobileNetV2)
# We don't include its final "top" layer
base_model = MobileNetV2(
    input_shape=input_shape,
    include_top=False, 
    weights='imagenet' # Load the knowledge from millions of images
)

# 2. FREEZE the expert's knowledge. 
# We don't want to re-train the part that knows about edges and textures.
base_model.trainable = False

# 3. Build your new model on top
model = models.Sequential([
    # Start with the frozen expert
    base_model,
    
    # Add our own classifier head
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3), # A dropout of 0.3 is a good starting point
    layers.Dense(num_classes, activation='softmax') # Your 29-class output
])

model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # <-- Perfect for integer labels!
    metrics=['accuracy']
)

In [ ]:
# Create a log directory for this specific run
# The datetime string makes each run unique in TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

# Create the TensorBoard callback
tensorboard_callback = callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1  # This logs weight histograms every epoch
)

In [ ]:
# Create a callback to stop training if validation accuracy doesn't improve
early_stopping = callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3, # Stop after 3 epochs of no improvement
    restore_best_weights=True # Automatically restore the best model weights
)

# Create a callback to save your best model to a file
model_checkpoint = callbacks.ModelCheckpoint(
    'model1.keras', # File name
    monitor='val_accuracy',
    save_best_only=True
)

In [ ]:
EPOCHS = 20 # Start with 20, EarlyStopping will stop it if it's done sooner

history = model.fit(
    train_ds,
    epochs=EPOCHS,
    validation_data=val_ds,
    callbacks=[early_stopping,
     model_checkpoint,
     tensorboard_callback
     ] # Pass in our helpers
)